In [31]:
import cv2
import mediapipe as mp
import numpy as np
import time

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands()

cap = cv2.VideoCapture(0)

tap_count_list = [] 
max_amplitude_list = []  
frozen_time_list = [] 

tapping_count = 0
prev_touching = False
max_amplitude = 0
frozen_start_time = None
threshold = 0.1

start_time = None  

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if start_time is None:  
        start_time = time.time()

    elapsed_time = time.time() - start_time
    if elapsed_time >= 5:
        break

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]
        thumb = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP]
        index_finger = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
        distance = np.linalg.norm(np.array([thumb.x, thumb.y]) - np.array([index_finger.x, index_finger.y]))

        if distance > max_amplitude:
            max_amplitude = distance
            frozen_start_time = time.time()

        touching = distance < threshold

        if not prev_touching and touching:
            tapping_count += 1
            tap_count_list.append(tapping_count)
            max_amplitude_list.append(max_amplitude)
            max_amplitude = 0

            if frozen_start_time is not None:
                frozen_time = time.time() - frozen_start_time
                frozen_time_list.append(frozen_time)
            frozen_start_time = None
        
        prev_touching = touching

        # Calculate bounding box coordinates
        x_min = int(min([lm.x * frame.shape[1] for lm in hand_landmarks.landmark]))
        y_min = int(min([lm.y * frame.shape[0] for lm in hand_landmarks.landmark]))
        x_max = int(max([lm.x * frame.shape[1] for lm in hand_landmarks.landmark]))
        y_max = int(max([lm.y * frame.shape[0] for lm in hand_landmarks.landmark]))

       
        cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    text_size = 0.5
    text_thickness = 1
    text_x = 10
    text_y = 20
    text_y_offset = 20

    cv2.putText(frame, f"Tapping Count: {tapping_count}", (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, text_size,
                (0, 255, 0), text_thickness)
    cv2.putText(frame, f"Amplitude: {max_amplitude:.2f}", (text_x, text_y + text_y_offset), cv2.FONT_HERSHEY_SIMPLEX,
                text_size, (0, 255, 0), text_thickness)
    cv2.imshow('Finger Tapping', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    elif cv2.waitKey(1) & 0xFF == ord('s'):  
        start_time = time.time()

cap.release()
cv2.destroyAllWindows()


for i in range(len(tap_count_list)):
    print(f"Tap {i+1}: Count = {tap_count_list[i]}, Max Amplitude = {max_amplitude_list[i]}, Frozen Time = {frozen_time_list[i]}")
avg_max_amp = sum(max_amplitude_list)/len(max_amplitude_list)
##keke
avg_frozen_time = sum(frozen_time_list)/len(frozen_time_list)
#find the highest tap
max_count = max(tap_count_list)
print(max_count,",",avg_max_amp, ",", avg_frozen_time)

Tap 1: Count = 1, Max Amplitude = 0.22653683575398115, Frozen Time = 0.16541695594787598
Tap 2: Count = 2, Max Amplitude = 0.11749643158358097, Frozen Time = 0.0808420181274414
Tap 3: Count = 3, Max Amplitude = 0.15338892610604116, Frozen Time = 0.1387169361114502
Tap 4: Count = 4, Max Amplitude = 0.23990768905847928, Frozen Time = 0.14049124717712402
Tap 5: Count = 5, Max Amplitude = 0.18275179758265403, Frozen Time = 0.15441012382507324
Tap 6: Count = 6, Max Amplitude = 0.10913614966061433, Frozen Time = 0.0772697925567627
Tap 7: Count = 7, Max Amplitude = 0.15322453877955902, Frozen Time = 0.06402349472045898
Tap 8: Count = 8, Max Amplitude = 0.180481146011606, Frozen Time = 0.1538994312286377
Tap 9: Count = 9, Max Amplitude = 0.11639873875140508, Frozen Time = 0.0639183521270752
Tap 10: Count = 10, Max Amplitude = 0.13660865149042034, Frozen Time = 0.14071059226989746
Tap 11: Count = 11, Max Amplitude = 0.2531748936635332, Frozen Time = 0.07902288436889648
Tap 12: Count = 12, Max A

In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

df = pd.read_csv('C:/Users/Kannan/Documents/FT/leftfeapark.csv')
X = df[['Count', 'Average Max Amplitude', 'Average Frozen Time']]
y = df['Level']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=688)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


smote = SMOTE(sampling_strategy='not majority', random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

model = DecisionTreeClassifier(max_depth=2)
model.fit(X_train_resampled, y_train_resampled)


y_pred = model.predict(X_test_scaled)

print(classification_report(y_test, y_pred))


class_distribution_resampled = pd.Series(y_train_resampled).value_counts()
# print("\nClass distribution in training set after SMOTE:")
# print(class_distribution_resampled)


              precision    recall  f1-score   support

           0       0.92      1.00      0.96        12
           1       0.96      0.81      0.88        27
           2       0.90      0.88      0.89        32
           3       0.43      1.00      0.60         3

    accuracy                           0.88        74
   macro avg       0.80      0.92      0.83        74
weighted avg       0.91      0.88      0.89        74



In [33]:

new_observation = [[max_count , avg_max_amp ,avg_frozen_time ]]

new_observation = scaler.transform(new_observation)

predicted_level = model.predict(new_observation)

print("The predicted level is:", predicted_level[0])
print(max_count, avg_max_amp,avg_frozen_time)

The predicted level is: 0
17 0.1627874975950157 0.11608158840852625


c:\Users\Kannan\anaconda3\envs\GG_3486\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
